In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))

import resspect
from resspect.tom_client import TomClient
from oracle_resspect_classifier.elasticc2_oracle_feature_extractor import ELAsTiCC2_ORACLEFeatureExtractor

/global/homes/a/arjun15/.conda/envs/resspect/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


In [ ]:
import itertools

# code to grab objects from TOM for testing - not the most optimal way to do this but a good check for whether the feature extractor is doing its job
username = 'arjun15'
passwordfile = '../../oracle/passwordfile'

detected_in_last_days = 1
mjd_now = 60800
num_objects = 5


tom = TomClient(url="https://desc-tom-2.lbl.gov", username=username, passwordfile=passwordfile)

dic = {
    'detected_in_last_days': detected_in_last_days,
    'mjd_now': mjd_now
}

res = tom.post('elasticc2/gethottransients', json=dic)
data = res.json() if res.status_code == 200 else {}
print('=> Fetched hot transients')

ids = [obj['objectid'] for obj in data['diaobject']]
ids = ids[:num_objects] if len(ids) > num_objects else ids

# using these object ids, load in static and time series data for ORACLE
static = tom.post('db/runsqlquery/',
                json={'query': '''SELECT diaobject_id, ra, decl, mwebv, mwebv_err, z_final, z_final_err, hostgal_zphot, hostgal_zphot_err,
                hostgal_zspec, hostgal_zspec_err, hostgal_ra, hostgal_dec, hostgal_snsep, hostgal_ellipticity, hostgal_mag_u,
                hostgal_mag_g, hostgal_mag_r, hostgal_mag_i, hostgal_mag_z, hostgal_mag_y FROM elasticc2_ppdbdiaobject WHERE diaobject_id IN (%s) ORDER BY diaobject_id;''' % (', '.join(str(id) for id in ids)),
                    'subdict': {}})
static_data = static.json() if static.status_code == 200 else {}
print('=> Loaded static data...')

ts = tom.post('db/runsqlquery/',
                json={'query': 'SELECT diaobject_id, midpointtai, filtername, psflux, psfluxerr FROM elasticc2_ppdbdiaforcedsource WHERE diaobject_id IN (%s) ORDER BY diaobject_id;' % (', '.join(str(id) for id in ids)),
                    'subdict': {}})
ts_data = ts.json() if ts.status_code == 200 else {}
print('=> Loaded time-series data...') 

assert ts_data['status'] == 'ok', 'Failed to retrieve data from TOM!'

# for each object, sort all observations by MJD
ts_data['rows'].sort(key=lambda obs: obs['diaobject_id'])
grouped_ts_data = {snid: list(obj) for snid, obj in itertools.groupby(ts_data['rows'], key=lambda obs: obs['diaobject_id'])}

for observation in grouped_ts_data.values():
    observation.sort(key=lambda obs: obs['midpointtai'])

print(ts_data)
print(static_data)

In [2]:
import polars as pl

nersc_parquet_files = '/global/cfs/cdirs/desc-td/ELASTICC2_TRAIN02_parquet'
parquet_example = os.path.join(nersc_parquet_files, 'SNIa-SALT3.parquet')
parquet = pl.read_parquet(parquet_example)

In [3]:
parquet["SNTYPE"]

SNTYPE
i32
110
110
110
10
110
…
110
10
10


In [7]:
example_input = parquet[0]

def get_phot_from_parquet(parquet_rows):
    class_to_sncode = {
        'SNIa': 10, 'SNIb/c': 25, 'SNII': 37, 'SNIax': 12, 'SN91bg': 11, 'KN': 50, 'M-dwarf Flare': 82, 'Dwarf Novae': 84, 'uLens': 88, 
        'SLSN': 40, 'TDE': 42, 'ILOT': 45, 'CART': 46, 'PISN': 59, 'Cepheid': 90, 'RR Lyrae': 80, 'Delta Scuti': 91, 'EB': 83, 'AGN': 60,
        'SNII': 32, 'SNII': 31, 'SNII': 35, 'SNII': 36, 'SNIb/c': 21, 'SNIb/c': 20, 'SLSN': 72, 'SNIb/c': 27, 'SNIb/c': 26
    }
    
    data = []
    for idx, obj in enumerate(parquet_rows.iter_rows(named=True)):        
        phot_d = {}
        phot_d['objectid'] = int(obj['SNID'])
        # phot_d['sncode'] = class_to_sncode[obj['ELASTICC_class']]
        phot_d['sncode'] = obj['SNTYPE']
        phot_d['redshift'] = obj['REDSHIFT_FINAL']
        phot_d['ra'] = obj['RA']
        phot_d['dec'] = obj['DEC']
        
        phot_d['photometry'] = {}
        phot_d['photometry']['band'] = obj['BAND']
        phot_d['photometry']['mjd'] = obj['MJD']
        phot_d['photometry']['midpointtai'] = obj['MJD']
        phot_d['photometry']['flux'] = obj['FLUXCAL']
        phot_d['photometry']['fluxerr'] = obj['FLUXCALERR']
        
        data.append(phot_d)
        
    return data

data_dic = get_phot_from_parquet(example_input)

In [8]:
data_dic

[{'objectid': 73,
  'sncode': 110,
  'redshift': 0.6541222929954529,
  'ra': 304.24339264195623,
  'dec': -4.291256140728584,
  'photometry': {'band': ['Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'Y',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'g',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i',
    'i

In [10]:
# testing out the feature extractor
from resspect.fit_lightcurves import fit, fit_TOM
feature_extraction_method = 'oracle_resspect_classifier.elasticc2_oracle_feature_extractor.ELAsTiCC2_ORACLEFeatureExtractor'
fit(
    data_dic,
    output_features_file = 'TOM_days_storage/TOM_training_features.csv',
    feature_extractor = feature_extraction_method,
    filters = 'LSST',
    # additional_info = additional_info,
    # one_code = gentypes
)

INFO:root:Starting oracle_resspect_classifier.elasticc2_oracle_feature_extractor.ELAsTiCC2_ORACLEFeatureExtractor fit...
/global/u2/a/arjun15/oracle-resspect-classifier/src/oracle_resspect_classifier/elasticc2_oracle_feature_extractor.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[60423.4061 60423.4145 60428.4206 60448.362  60483.3557 60483.3786
 60487.3155 60506.1956 60506.201  60506.2241 60562.0093 60562.0325
 60565.0684 60587.9941 60588.0037 60599.044  60807.3743 60807.3843
 60808.3566 60849.3256 60849.3485 60859.2219 60862.1836 60862.2069
 60865.3052 60870.232  60897.2459 60899.1459 60913.0834 60913.106
 60918.1063 60920.149  60930.0683 60931.0282 60931.0496 60931.0735
 60940.0217 60953.0909 60964.9983 60964.9992 60965.0085 60965.0094
 60966.9984 60967.0084 60981.103  60490.2464 60503.2684 60525.1367
 60528.1232 60528.1237 60558.057  60855.1631 60856.3019 60873.1872
 60877.2159 60904.1705 6090

ValueError: setting an array element with a sequence.

In [ ]:
# viewing the output
import numpy as np

data = pl.read_csv('TOM_days_storage/TOM_training_features.csv', index_col=False)
data['orig_sample'] = 'train'
data['type'] = np.where(data['sncode'] == 10 or data['sncode'] == 110, 'Ia', 'other')
data.to_csv('TOM_days_storage/TOM_training_features.csv', index=False)

data